In [3]:
import pandas as pd
import numpy as np
from scipy.stats import chi2_contingency, chi2, fisher_exact
from scipy.stats import kstest, norm

In [4]:
test_donnees=pd.read_csv('test_données.csv')

/var/folders/y0/0nrj3m412p978185q3d2503sr02q24/T/ipykernel_39585/426646326.py:1: DtypeWarning: Columns (16,50,51,55,56,65,67,69,77,84,85,86,87,88,89,118) have mixed types. Specify dtype option on import or set low_memory=False.
  test_donnees=pd.read_csv('test_données.csv')


In [5]:
donnees=test_donnees.loc[test_donnees['Vehicle'].isin(['E-bike','Bike','E-scooter'])]


In [6]:
donnees=donnees.loc[donnees['severity']!=-1]


In [7]:
donnees

,Unnamed: 0,Num_Acc,day,month,Year,hrmn,lum,com,int,atm,...,severity_3,Point of impact_3,id_vehicule_opposite_3,age_opposite_mean,age_driver,Helmet_driver,Gender_driver,severity_driver,Age category involved,Accident location
1,1,2.022000e+11,21,10,2022,16:32,1,75106,1,1,...,NaN,NaN,NaN,62.0,999.0,999,999,999.0,61+,On road
3,3,2.022000e+11,20,10,2022,13:00,1,75105,2,1,...,NaN,NaN,NaN,20.0,999.0,999,999,999.0,21-40,On cycle facility
4,4,2.022000e+11,21,10,2022,11:25,1,75113,4,1,...,NaN,NaN,NaN,0.0,999.0,999,999,999.0,0-20,On cycle facility
6,6,2.022000e+11,21,10,2022,15:50,1,75103,2,1,...,NaN,NaN,NaN,32.0,999.0,999,999,999.0,21-40,On road
7,7,2.022000e+11,21,10,2022,19:40,5,93049,2,1,...,NaN,NaN,NaN,48.0,999.0,999,999,999.0,41-60,On road
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25650,25650,2.023001e+11,14,7,2023,00:15,5,69385,1,1,...,NaN,NaN,NaN,20.0,999.0,999,999,999.0,21-40,On other special way
25651,25651,2.023001e+11,14,7,2023,00:15,5,69385,1,1,...,NaN,NaN,NaN,20.0,21.0,No,Male,2.0,21-40,On other special way
25654,25654,2.023001e+11,16,6,2023,15:20,1,69381,1,1,...,NaN,NaN,NaN,44.0,999.0,999,999,999.0,41-60,On cycle facility
25656,25656,2.023001e+11,13,7,2023,07:50,1,69266,2,1,...,NaN,NaN,NaN,53.0,999.0,999,999,999.0,41-60,On road


In [8]:
data_acc=donnees.drop_duplicates('Num_Acc')

In [9]:
var_for_acc=['Num_Acc', 'jour', 'mois', 'an', 'hrmn', 'lum', 'dep', 'com', 'agg',
       'int', 'atm', 'col', 'adr', 'lat', 'long', 'geometry', 'circ', 'nbv', 'vosp', 'prof',
        'plan', 'lartpc', 'larrout', 'surf', 'infra', 'situ',
       'Weekend',
            'Year', 'Time category','Lighting conditions','Weather conditions', 'Involved vehicle 1 type', 
          'Point of impact', 'Reglementation',  'Cycle facilities', 'Agglomeration', 'Max speed',
        'Road type', 'Intersection', 'Crossroad','number of involved vehicles',
        
      'Long profile', 'Truck traffic',
       'Pavement',  'Road width',
       'Surface condition', 'Time of day']

var_cont=['age','age_2','Number of passengers','number of involved vehicles','vma','largeurcha','pentemoyen','largeurtro']

In [10]:
data_acc

,Unnamed: 0,Num_Acc,day,month,Year,hrmn,lum,com,int,atm,...,severity_3,Point of impact_3,id_vehicule_opposite_3,age_opposite_mean,age_driver,Helmet_driver,Gender_driver,severity_driver,Age category involved,Accident location
1,1,2.022000e+11,21,10,2022,16:32,1,75106,1,1,...,NaN,NaN,NaN,62.0,999.0,999,999,999.0,61+,On road
3,3,2.022000e+11,20,10,2022,13:00,1,75105,2,1,...,NaN,NaN,NaN,20.0,999.0,999,999,999.0,21-40,On cycle facility
4,4,2.022000e+11,21,10,2022,11:25,1,75113,4,1,...,NaN,NaN,NaN,0.0,999.0,999,999,999.0,0-20,On cycle facility
6,6,2.022000e+11,21,10,2022,15:50,1,75103,2,1,...,NaN,NaN,NaN,32.0,999.0,999,999,999.0,21-40,On road
7,7,2.022000e+11,21,10,2022,19:40,5,93049,2,1,...,NaN,NaN,NaN,48.0,999.0,999,999,999.0,41-60,On road
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25649,25649,2.023001e+11,23,8,2023,15:30,1,69266,2,1,...,NaN,NaN,NaN,29.0,999.0,999,999,999.0,21-40,On cycle facility
25650,25650,2.023001e+11,14,7,2023,00:15,5,69385,1,1,...,NaN,NaN,NaN,20.0,999.0,999,999,999.0,21-40,On other special way
25654,25654,2.023001e+11,16,6,2023,15:20,1,69381,1,1,...,NaN,NaN,NaN,44.0,999.0,999,999,999.0,41-60,On cycle facility
25656,25656,2.023001e+11,13,7,2023,07:50,1,69266,2,1,...,NaN,NaN,NaN,53.0,999.0,999,999,999.0,41-60,On road


In [11]:
# Fonction pour calculer le Chi-carré et p-value pour chaque catégorie avec les règles de Cochran
import pandas as pd
import numpy as np
from scipy.stats import chi2_contingency, fisher_exact, f_oneway,  kruskal, shapiro

def calculate_chi2_p_value_for_each_category(data, var):
    categories = data[var].unique()
    p_values = {}
    for category in categories:
        # Créer une table de contingence pour la catégorie
        contingency_table = pd.crosstab(data['severity'], data[var] == category)
        
        # Effectuer le test du Chi-carré
        chi2, p, _, expected = chi2_contingency(contingency_table)
        
        # Appliquer les règles de Cochran
        if np.any(expected < 1) or (np.sum(expected < 5) / expected.size) > 0.2:
            p_values[category] = np.nan  # Indicate that the test is not valid

        else:
            # Initialize p-value formatting
            p_value_str = '<0.001' if p < 0.001 else str(round(p, 3))
            
            # Add significance markers
            if p < 0.01 and p > 0.001:
                p_value_str += ''
            elif p < 0.05 and p > 0.01:
                p_value_str += ''
            
            # Store the result
            p_values[category] = p_value_str
    return p_values


# Fonction pour compter par sévérité et variable d'intérêt
def count_by_severity(data, var):
    # Groupement par 'severity' et la variable d'intérêt
    counts = data.groupby(['severity', var]).size().astype(int).reset_index(name='count')
   
    total_counts = counts.groupby(var)['count'].transform('sum')
    counts['percentage'] = (counts['count'] / total_counts * 100).round(2)
    
    # Convertir 'count' en entier pour éviter les .0
    counts['count'] = counts['count'].astype(int)
    
    counts['count_percentage'] = counts.apply(lambda row: f"{row['count']} ({row['percentage']}%)", axis=1)
    return counts


def compare_continuous_variable(data, group_var, continuous_var):

    
    groups_data = [data[data[group_var] == group][continuous_var].dropna().values for group in data[group_var].unique()]
    

    normal = all(kstest(group, 'norm', args=(group.mean(), group.std())).pvalue > 0.05 for group in groups_data)

    
    if normal:
        # Essayer ANOVA
        f_stat, p_anova = f_oneway(*groups_data)
        test_used = 'ANOVA'
        p = p_anova
    else:
        # Si les données ne sont pas normales, faire un test de Kruskal-Wallis
        h_stat, p_kruskal = kruskal(*groups_data)
        test_used = 'Kruskal-Wallis'
        p = p_kruskal
    
    # Initialiser la mise en forme de la p-value
    if p < 0.001:
        p_value_str = '<0.001'
    else:
        p_value_str = str(round(p, 3))
    
    if p < 0.01 and p > 0.001:
        p_value_str += ''
    elif p < 0.05 and p > 0.01:
        p_value_str += ''
    
    return f'{p_value_str}'

# Exemple de variables catégorielles
variables = [
    'Age category', 'Gender', 'Vehicle', 'User category', 'Helmet', 
    'Reflective jacket', 'Lighting conditions', 
    'Weather conditions', 'Point of impact', 'Maneuver',
'Cycle facilities', 'Accident location', 'Trip purpose',
   'Max speed', 'Intersection', 'Crossroad', 
    'Long profile', 'Pavement', 'Surface condition','Road width', 'vehicle_type_2', 'Maneuver_2', 'Gender_2','Age category involved', 'tram','positionnement_piste',
    
'Road type'
]


pivot_results_dfs = []

# Compter les occurrences pour chaque variable catégorielle et stocker les résultats pivotés
for var in variables:
    counts_df = count_by_severity(data_acc if var in var_for_acc else donnees, var)
    pivot_df = counts_df.pivot(index=var, columns='severity', values='count_percentage').reset_index()
    pivot_df['Variable'] = var  # Ajouter une colonne pour la variable
    pivot_df['cat'] = pivot_df[var]
    pivot_df.drop(columns=var, inplace=True)
    
    # Calculer la p-value du Chi-carré pour chaque catégorie
    p_values = calculate_chi2_p_value_for_each_category(data_acc if var in var_for_acc else donnees, var)
    pivot_df['p_value'] = pivot_df['cat'].map(p_values)
    
    pivot_results_dfs.append(pivot_df)

# Ajouter les variables continues avec leurs moyennes et p-values
for var in var_cont:
    df_to_use = data_acc if var in var_for_acc else donnees
    df_to_use = df_to_use[['severity', var]].dropna(subset=[var])
    
    # Calcul de la médiane et de l'IQR
    stats_df = df_to_use.groupby('severity')[var].agg(
        median='median',
        q1=lambda x: x.quantile(0.25),
        q3=lambda x: x.quantile(0.75)
    ).reset_index()
    
    # Format : "médiane [Q1–Q3]"
    stats_df['median_iqr'] = stats_df.apply(
        lambda row: f"{row['median']:.2f} [{row['q1']:.2f}–{row['q3']:.2f}]", axis=1
    )
    
    # Réorganisation du tableau
    stats_df.drop(columns=['median', 'q1', 'q3', 'severity'], inplace=True)
    stats_df = stats_df.T
    stats_df.columns = [1, 2, 3]
    stats_df['Variable'] = var

   

    
    # Calculer la p-value de l'ANOVA
    p_value_str = compare_continuous_variable(data_acc if var in var_for_acc else donnees, 'severity', var)
    stats_df['p_value'] = p_value_str
    
    pivot_results_dfs.append(stats_df)

# Concaténer tous les résultats pivotés en un seul DataFrame
final_pivot_df = pd.concat(pivot_results_dfs, ignore_index=True)

# Ordonner les colonnes
column_order = ['Variable', 'cat'] + [col for col in final_pivot_df.columns if col not in ['Variable', 'cat']]
final_pivot_df = final_pivot_df[column_order]
final_pivot_df = final_pivot_df.replace({
    'vehicle_type_2': 'Type of the third-party vehicle',
    'Maneuver_2': 'Maneuver of the third-party vehicle',
    'Gender_2': 'Gender of the third-party vehicle',
    'Point of impact_opposite':'Impact of the third-party vehicle',
    'vma':'Car speed limit',
    'largeurcha':'Road width',
    'pentemoyen':'Avergae slope',
    'largeurtro':'Sidewalk width',
    'age':'Age of the individual',
    'number of involved vehicles':'Number of involved vehicles in the crash'
})


In [12]:
final_pivot_df

,Variable,cat,1,2,3,p_value
0,Age category,0-20,324 (17.49%),1520 (82.07%),8 (0.43%),0.002
1,Age category,21-40,1103 (14.98%),6227 (84.55%),35 (0.48%),0.042
2,Age category,41-60,354 (9.48%),3352 (89.72%),30 (0.8%),<0.001
3,Age category,61+,62 (6.01%),951 (92.24%),18 (1.75%),<0.001
4,Gender,Female,330 (7.89%),3832 (91.57%),23 (0.55%),<0.001
...,...,...,...,...,...,...
118,Number of involved vehicles in the crash,NaN,2.00 [2.00–2.00],2.00 [2.00–2.00],2.00 [1.00–2.00],<0.001
119,Car speed limit,NaN,30.00 [30.00–50.00],50.00 [30.00–50.00],50.00 [30.00–50.00],<0.001
120,Road width,NaN,8.10 [6.00–10.20],8.00 [6.20–10.20],7.35 [6.20–9.05],0.374
121,Avergae slope,NaN,1.00 [1.00–1.00],1.00 [1.00–2.00],1.00 [1.00–2.50],<0.001


In [13]:
import pandas as pd

def generate_latex_table(df):
    def escape_latex(value):
        """Escape special LaTeX characters in a string."""
        if isinstance(value, str):
            return value.replace('%', r'\%').replace('_', r'\_').replace('&', r'\&')
        return value

    latex_code = "\\begin{longtable}{p{4.5cm}p{3cm}p{3cm}p{3cm}p{1.5cm}}\n"
    latex_code += "\\caption{Descriptive statistics of the dataset} \\label{tab:data} \\\\\n"
    latex_code += "\\hline\n"
    latex_code += "\\textbf{Category} & \\textbf{No injured} & \\textbf{Injured} & \\textbf{Killed} & \\textbf{p-value} \\\\\n"
    latex_code += "\\hline\n"
    latex_code += "\\endfirsthead\n\n"
    latex_code += "\\hline\n"
    latex_code += "\\textbf{Category} & \\textbf{No injured} & \\textbf{Injured} & \\textbf{Killed} & \\textbf{p-value}  \\\\\n"
    latex_code += "\\hline\n"
    latex_code += "\\endhead\n\n"
    latex_code += "\\hline\n"
    latex_code += "\\endfoot\n\n"
    latex_code += "\\hline\n"
    latex_code += "\\endlastfoot\n"

    previous_variable = None

    for _, row in df.iterrows():
        # Escape LaTeX special characters
        variable = escape_latex(row['Variable']) if pd.notna(row['Variable']) else ""
        cat = escape_latex(row['cat']) if pd.notna(row['cat']) else ""
        no_injured = escape_latex(row[1]) if pd.notna(row[1]) else ""
        injured = escape_latex(row[2]) if pd.notna(row[2]) else ""
        killed = escape_latex(row[3]) if pd.notna(row[3]) else ""
        p_value = escape_latex(row['p_value']) if pd.notna(row['p_value']) else ""

        # Check if the variable name has changed
        if variable != previous_variable:
            if previous_variable is not None:
                latex_code += "\\hline\n"  # Add \hline before a new variable starts
            previous_variable = variable
            latex_code += f"\\multicolumn{{5}}{{c}}{{\\textbf{{{variable}}}}} \\\\\n"

        # Append the row to the LaTeX code
        latex_code += f"{cat} & {no_injured} & {injured} & {killed} & {p_value}  \\\\\n"

    latex_code += "\\end{longtable}"

    return latex_code

# Example usage
# Assuming `final_pivot_df` is your DataFrame
latex_table = generate_latex_table(final_pivot_df)
# print(latex_table)


In [14]:
# Sauvegarder le code LaTeX dans un fichier .tex
with open("table_latex_output.tex", "w") as f:
    f.write(latex_table)

print("Le code LaTeX a été enregistré dans 'table_latex_output.tex'.")


Le code LaTeX a été enregistré dans 'table_latex_output.tex'.


In [15]:
var_added=pd.get_dummies(donnees[['Num_Acc','Cycle facilities','Pavement','Crossroad','positionnement_piste','Road type','Reglementation','tram']])
# Assuming data_acc and var_added are already defined DataFrames
var_added = var_added.astype(int)  # Convert boolean to integers


# Merge the 'Max speed' column from data_acc into var_added
var_added = var_added.merge(data_acc[['Num_Acc', 'largeurcha','pentemoyen','largeurtro']], on='Num_Acc', how='left')

# Display the updated var_added DataFrame
var_added

,Num_Acc,Cycle facilities_Bus lane,Cycle facilities_Cycle lane,Cycle facilities_Cycle path/Greenway,Cycle facilities_No cycle facilities,Cycle facilities_Pedestrianized street,Pavement_Asphalt,Pavement_Concrete,Pavement_Paved,Crossroad_No intersection,...,Reglementation_Meeting zone (Z20),Reglementation_No particularity,Reglementation_Pedestrian area,Reglementation_Zone 30,tram_No,tram_On tracks,tram_Tram crossing,largeurcha,pentemoyen,largeurtro
0,202200000016,0,0,0,1,0,1,0,0,1,...,0,0,0,0,1,0,0,NaN,NaN,NaN
1,202200000017,0,0,0,1,0,1,0,0,0,...,0,0,0,0,1,0,0,NaN,NaN,NaN
2,202200000025,0,0,1,0,0,1,0,0,0,...,0,0,0,0,0,0,1,NaN,NaN,NaN
3,202200000027,0,0,0,1,0,1,0,0,0,...,0,0,0,0,1,0,0,NaN,NaN,NaN
4,202200000046,0,0,0,1,0,1,0,0,0,...,0,0,0,0,1,0,0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14271,202300054715,1,0,0,0,0,0,0,0,1,...,0,1,0,0,1,0,0,NaN,NaN,NaN
14272,202300054715,1,0,0,0,0,0,0,0,1,...,0,1,0,0,1,0,0,NaN,NaN,NaN
14273,202300054723,0,0,1,0,0,1,0,0,1,...,0,0,0,1,1,0,0,2.7,2.0,4.0
14274,202300054787,0,1,0,0,0,1,0,0,0,...,0,0,0,1,1,0,0,10.6,0.0,4.8


In [16]:
# Assuming var_added is already defined as a DataFrame

# Initialize lists to store the results
columns = []
means = []
mins = []
maxs = []

# Calculate mean, min, and max for each column in var_added
for var in var_added.columns:
    mean_value = var_added[var].mean()
    min_value = var_added[var].min()
    max_value = var_added[var].max()
    
    columns.append(var)
    means.append(mean_value)
    mins.append(min_value)
    maxs.append(max_value)

# Create a DataFrame to hold the results
results_df = pd.DataFrame({
    'Column': columns,
    'Mean': means,
    'Min': mins,
    'Max': maxs
})

# Display the DataFrame
results_df

,Column,Mean,Min,Max
0,Num_Acc,2.021246e+11,2.019000e+11,2.023001e+11
1,Cycle facilities_Bus lane,1.239843e-01,0.000000e+00,1.000000e+00
2,Cycle facilities_Cycle lane,1.330905e-01,0.000000e+00,1.000000e+00
3,Cycle facilities_Cycle path/Greenway,1.645419e-01,0.000000e+00,1.000000e+00
4,Cycle facilities_No cycle facilities,5.447604e-01,0.000000e+00,1.000000e+00
5,Cycle facilities_Pedestrianized street,3.362286e-02,0.000000e+00,1.000000e+00
6,Pavement_Asphalt,8.449145e-01,0.000000e+00,1.000000e+00
7,Pavement_Concrete,8.615859e-03,0.000000e+00,1.000000e+00
8,Pavement_Paved,5.204539e-02,0.000000e+00,1.000000e+00
9,Crossroad_No intersection,3.748249e-01,0.000000e+00,1.000000e+00


In [17]:
# Liste des niveaux de severité que vous voulez comparer
severities = [1, 2, 3]

# DataFrame pour stocker les p-values
p_values_df = pd.DataFrame(columns=[f'severity={s}' for s in severities if s != 1], index=var_cont)

# Effectuer le test KS pour chaque variable continue
for var in var_cont:

    # Séparer les données en fonction de la variable 'severity' pour la variable continue actuelle
    grouped_data = {s: donnees[donnees['severity'] == s][var].dropna().values for s in severities}
    
    # Données de référence (severity=1)
    ref_group = grouped_data[1]
    
    for sev in severities:
        if sev == 1:
            continue
        # Comparer le groupe actuel à la référence 'severity=1'
        test_group = grouped_data[sev]
        _, p_value = ks_2samp(test_group, ref_group)
        p_value=p_value.round(3)
        p_values_df.loc[var, f'severity={sev}'] = p_value


print(p_values_df)

NameError: name 'ks_2samp' is not defined

In [ ]:
p_values_df.to_csv('twosample_ks.csv',sep=';')